In [ ]:
import xarray as xr
import numpy as np

In [ ]:
input_dir = '/g/data/er4/vd5822/files_to_check/mrnbc_speckling/input/'
output_dir = '/g/data/er4/vd5822/files_to_check/mrnbc_speckling/output/'

input_file = input_dir + 'pr_mrnbc_CNRM-CM5_rcp85.nc4'

In [ ]:
ds = xr.open_dataset(input_file)

In [ ]:
pr_threshold = 0.009259
# pr 0,6,145

specks = ds.where(ds > pr_threshold)
specks_removed = ds.where(ds.pr < pr_threshold)

In [ ]:
ds['pr'][0,6,145]

In [ ]:
specks_removed['pr'][0,6,145]

In [ ]:
interpolated_on_lat = specks_removed.interpolate_na(dim='lat')
interpolated_on_lon = specks_removed.interpolate_na(dim='lon')

In [ ]:
p = interpolated_on_lat*0.5 + interpolated_on_lon*0.5

In [ ]:
z = ds.where(ds < pr_threshold, p)

In [ ]:
z.to_netcdf(output_dir + 'pr_mrnbc_CNRM-CM5_rcp85_merge_interpna_lat_lon.nc4')

In [ ]:
lats = ds['lat'].data
lons = ds['lon'].data

count = 0
for lati,lat in np.ndenumerate(lats):
    for loni,lon in np.ndenumerate(lons):
        if ds['pr'][0,lati[0],loni[0]].data > pr_threshold:
            d = ds.interp(lat=lat, lon=lon)['pr'].data
            ds['pr'][:,lati[0],loni[0]] = d
            count +=1
            print(count)

In [ ]:
ds.to_netcdf(output_dir + 'pr_mrnbc_CNRM-CM5_rcp85_naive_interp.nc4')

In [ ]:
ds.close()